In [ ]:
import pandas as pd
import ast
import numpy as np
from scipy.sparse import lil_matrix
from collections import Counter
import scipy.sparse
import pickle

In [ ]:
ways = pd.read_csv('way_list.csv',sep='|')
nodes = pd.read_csv('node_list.csv',sep='|')
nodes.set_index('id',inplace=True)
ways['nodes'] = ways['nodes'].apply(lambda x: ast.literal_eval(x))
nodes = nodes.drop_duplicates()

In [ ]:
# Inefficient code
# all_nodes = set(ways['nodes'].sum())
# # all_nodes = ways['nodes'].sum()

# node_tracker = {}
# counter = 0

# # Here we are getting all_nodes_count
# for i in all_nodes:
#     node_tracker[i] = counter
#     counter +=1

# # Creating the matrix
# list_of_all_lists = []
# for i in range(counter):
#     list_of_all_lists.append([[] for x in range(counter)])

# # main_list_of_all_lists
# main_list_of_all_lists = []
# for i in range(counter):
#     main_list_of_all_lists.append([[] for x in range(counter)])

# # tagging the segments in the matrix
# segment_tracker = 0
# for node in ways['nodes']:
#     for node_counter in range(len(node)-1):
#         list_of_all_lists[node_tracker[node[node_counter]]][node_tracker[node[node_counter+1]]].append(segment_tracker)
#         segment_tracker += 1


all_nodes_d = ways['nodes'].sum()
total_nodes = len(all_nodes_d)
all_nodes = set(all_nodes_d)
node_tracker = {}
counter = 0

# Here we are getting all_nodes_count
for i in all_nodes:
    node_tracker[i] = counter
    counter +=1

# Creating the matrix
list_of_all_lists = lil_matrix((total_nodes,total_nodes))

# Lets get all the intersections            
counter = Counter(all_nodes_d)

all_intersections = {}
for element in counter:
    if counter[element] > 1:
        all_intersections[element] = 0

segment_tracker = 0
for nodes_counter in range(len(ways)):
    if nodes_counter % 100 == 0:
        print(nodes_counter)
    intersections = set(ways['nodes'][nodes_counter]).intersection(all_intersections)

    for node_counter in range(len(ways['nodes'][nodes_counter])-1):
        if (ways['nodes'][nodes_counter][node_counter] in intersections):
            segment_tracker += 1
   
        list_of_all_lists[node_tracker[ways['nodes'][nodes_counter][node_counter]], node_tracker[ways['nodes'][nodes_counter][node_counter+1]]] = segment_tracker

    segment_tracker+=1

    
with open(r"sparse_matrix.pickle", "wb") as output_file:
    pickle.dump(list_of_all_lists, output_file)

all_sets = []
counter = 0
for nodes_counter in range(len(ways)):
    node_ids = set()
    
    
    for node_counter in range(len(ways['nodes'][nodes_counter])-1):
        set_ids = {}
        from_ = ways['nodes'][nodes_counter][node_counter]
        to_ = ways['nodes'][nodes_counter][node_counter+1]
        segment_id = list_of_all_lists[node_tracker[from_],node_tracker[to_]]
        
        
        set_ids['segment_id'] = segment_id
        set_ids['node1'] = from_
        set_ids['node2'] = to_
        set_ids['index'] = counter
        counter+=1
        # print(counter)
        all_sets.append(set_ids)
    # break
    del set_ids

main_df = pd.DataFrame(all_sets)

def get_lat(x):
    try:
        return nodes['lat'][x]
    except:
        return 'null'

def get_lon(x):
    try:
        return nodes['lon'][x]
    except:
        return 'null'

main_df['node1_lat'] = main_df['node1'].apply(get_lat)
main_df['node1_lon'] = main_df['node1'].apply(get_lon)
main_df['node2_lat'] = main_df['node2'].apply(get_lat)
main_df['node2_lon'] = main_df['node2'].apply(get_lon)

main_df.to_excel('Segment_wise_data.xlsx',index=False)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [ ]:
main_df

,segment_id,node1,node2,index,node1_lat,node1_lon,node2_lat,node2_lon
0,2156.0,9021077061,111316294,0,null,null,null,null
1,2157.0,111316294,9021077121,1,null,null,43.024263,-78.821124
2,2158.0,9021077121,111316298,2,43.024263,-78.821124,null,null
3,2159.0,111316298,111316300,3,null,null,null,null
4,2160.0,111316300,7088597800,4,null,null,null,null
...,...,...,...,...,...,...,...,...
257748,198196.0,7788014816,7788014815,257748,42.880404,-78.733187,42.880175,-78.733626
257749,198198.0,941291086,8556196216,257749,42.882691,-78.731363,42.882756,-78.731325
257750,198198.0,8556196216,8556196214,257750,42.882756,-78.731325,42.882764,-78.731313
257751,198198.0,8556196214,8556196215,257751,42.882764,-78.731313,42.882764,-78.731297


In [ ]:
set_ids

{'node1': 8556196215, 'node2': 941291107, 'segment_id': 198198.0}

In [ ]:
nodes['lat'][111316320]

43.0225328